In [188]:
import pandas as pd

def clean(week_nums):
    dfs = []
    for week_num in week_nums:
        url = "http://web.mta.info/developers/data/nyct/turnstile/turnstile_{}.txt"
        file_url = url.format(week_num)
        names = ['c_a', 'unit', 'scp', 'station', 'linename', 'division', 
                 'date', 'time', 'desc', 'entries', 'exits']
        df = pd.read_csv(file_url, names=names, parse_dates=[['date','time']], 
                         keep_date_col=True, skiprows=1)
        df['date'] = pd.to_datetime(df['date'])
        df['day_of_week'] = df['date_time'].dt.day_name()
        df = df[(~df['c_a'].str.contains('PTH') & 
                 ~df['desc'].str.contains('RECOVR') & 
                 df.time.astype(str).str.contains('00:00'))]
        df = df[['station', 'unit', 'c_a', 'scp', 'date_time', 'date', 'day_of_week', 'time', 
                'desc', 'entries', 'exits']]
        #delete duplicates -> could be done after dataframe creation instead of in this loop
        df.sort_values(['c_a', 'unit', 'scp', 'station', 'date_time'], inplace=True, ascending=False)
        df.drop_duplicates(subset=['c_a', 'unit', 'scp', 'station', 'date_time'], inplace=True)
        dfs.append(df)
        print(week_num)
    return pd.concat(dfs)

week_nums = [160903, 160910, 160917]

In [189]:
turnstiles_df = clean(week_nums)

160903
160910
160917


In [197]:
turnstiles_df.date


193138   2016-09-02
193137   2016-09-02
193136   2016-09-02
193135   2016-09-02
193134   2016-09-02
            ...    
4        2016-09-10
3        2016-09-10
2        2016-09-10
1        2016-09-10
0        2016-09-10
Name: date, Length: 532709, dtype: datetime64[ns]

In [173]:
turnstiles_block = (turnstiles_df
                        .groupby(["c_a", "unit", "scp", "station", "date_time", "exits"],as_index=False).entries.first())
turnstiles_block[["prev_datetime", "prev_entries", "prev_exits"]] = (turnstiles_block
                                                       .groupby(["c_a", "unit", "scp", "station"])["date_time", "entries", "exits"]
                                                       .apply(lambda grp: grp.shift(1)))
turnstiles_block.dropna(subset=["prev_datetime"], axis=0, inplace=True)
def get_counts_entry(row, max_counter):
    counter = row["entries"] - row["prev_entries"]
    if counter < 0:
        # Maybe counter is reversed?
        counter = -counter
    if counter > max_counter:
        # Maybe counter was reset to 0? 
        print(row["entries"], row["prev_entries"])
        counter = min(row["entries"], row["prev_entries"])
    if counter > max_counter:
        # Check it again to make sure we're not still giving a counter that's too big
        return 0
    return counter
def get_counts_exit(row, max_counter):
    counter = row["exits"] - row["prev_exits"]
    if counter < 0:
        # Maybe counter is reversed?
        counter = -counter
    if counter > max_counter:
        # Maybe counter was reset to 0? 
        print(row["exits"], row["prev_exits"])
        counter = min(row["exits"], row["prev_exits"])
    if counter > max_counter:
        # Check it again to make sure we're not still giving a counter that's too big
        return 0
    return counter

<ipython-input-173-7112b57ef14a>:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  turnstiles_block[["prev_datetime", "prev_entries", "prev_exits"]] = (turnstiles_block


In [174]:
turnstiles_block['delta_entries'] = turnstiles_block.apply(get_counts_entry, axis=1, max_counter=1e5)

29 139012.0
148 1494569141.0
2214 14129759.0
335599567 1374824.0
186916225 604712476.0
329 11078609.0
1794 239660.0
1820 9170515.0
76 564040.0


In [175]:
turnstiles_block['delta_exits'] = turnstiles_block.apply(get_counts_exit, axis=1, max_counter=1e5)
turnstiles_block

2 108703.0
19 122365.0
54 135367424.0
58 12723414.0
201390933 803021.0
2 6881370.0
835479154 869165449.0
247 6942597.0
46 102417.0
24 7344161.0
1458032 1326851.0
1327032 1458032.0
164 1021615.0


,c_a,unit,scp,station,date_time,exits,entries,prev_datetime,prev_entries,prev_exits,delta_entries,delta_exits
1,A002,R051,02-00-00,59 ST,2016-08-27 04:00:00,1966044,5799463,2016-08-27 00:00:00,5799442.0,1966041.0,21.0,3.0
2,A002,R051,02-00-00,59 ST,2016-08-27 08:00:00,1966079,5799492,2016-08-27 04:00:00,5799463.0,1966044.0,29.0,35.0
3,A002,R051,02-00-00,59 ST,2016-08-27 12:00:00,1966155,5799610,2016-08-27 08:00:00,5799492.0,1966079.0,118.0,76.0
4,A002,R051,02-00-00,59 ST,2016-08-27 16:00:00,1966214,5799833,2016-08-27 12:00:00,5799610.0,1966155.0,223.0,59.0
5,A002,R051,02-00-00,59 ST,2016-08-27 20:00:00,1966271,5800121,2016-08-27 16:00:00,5799833.0,1966214.0,288.0,57.0
...,...,...,...,...,...,...,...,...,...,...,...,...
177192,TRAM2,R469,00-05-01,RIT-ROOSEVELT,2016-09-02 05:00:00,259,5554,2016-09-02 01:00:00,5554.0,259.0,0.0,0.0
177193,TRAM2,R469,00-05-01,RIT-ROOSEVELT,2016-09-02 09:00:00,259,5554,2016-09-02 05:00:00,5554.0,259.0,0.0,0.0
177194,TRAM2,R469,00-05-01,RIT-ROOSEVELT,2016-09-02 13:00:00,259,5554,2016-09-02 09:00:00,5554.0,259.0,0.0,0.0
177195,TRAM2,R469,00-05-01,RIT-ROOSEVELT,2016-09-02 17:00:00,259,5554,2016-09-02 13:00:00,5554.0,259.0,0.0,0.0


In [176]:
# mf = turnstiles_block.groupby(['unit', 'station', 'date_time',]).agg({'delta_exits': 'sum'}).reset_index()
# mf[:50]

In [177]:



# top10 = (mf.sort_values(by='delta_exits',ascending=False).station.head(10))
# #pd.DataFrame(top10)
# top10

In [198]:
#unit vs Station name
#station complexes vs units vs sationts?

unit_hourly = (turnstiles_block.groupby(['unit','date_time'])['delta_exits'].sum().reset_index())

In [199]:
unit_hourly[:50]

,unit,date_time,delta_exits
0,R001,2016-08-27 05:00:00,500.0
1,R001,2016-08-27 09:00:00,1557.0
2,R001,2016-08-27 13:00:00,8168.0
3,R001,2016-08-27 17:00:00,6214.0
4,R001,2016-08-27 21:00:00,4138.0
5,R001,2016-08-28 01:00:00,2377.0
6,R001,2016-08-28 05:00:00,604.0
7,R001,2016-08-28 09:00:00,1115.0
8,R001,2016-08-28 13:00:00,6011.0
9,R001,2016-08-28 17:00:00,5675.0


In [239]:
top = unit_hourly.groupby(['unit', 'date_time'])['delta_exits'].sum().reset_index()

        

top10 = (top.sort_values(by=['delta_exits'],ascending=False)
        .unit.head(100))

In [240]:
top10.unique()

array(['R020', 'R170', 'R084', 'R021', 'R032', 'R011', 'R046', 'R022',
       'R055', 'R017', 'R179', 'R023', 'R012', 'R028', 'R293', 'R031',
       'R018'], dtype=object)

In [236]:
top10.head(100)

767     R020
761     R020
755     R020
749     R020
773     R020
6495    R170
6482    R170
6470    R170
6476    R170
6488    R170
3009    R084
6478    R170
6472    R170
802     R021
1206    R032
1200    R032
3003    R084
2997    R084
1194    R032
6484    R170
6483    R170
796     R021
3015    R084
403     R011
1776    R046
1770    R046
1782    R046
6494    R170
6477    R170
843     R022
1764    R046
1212    R032
808     R021
6490    R170
6489    R170
2139    R055
391     R011
849     R022
837     R022
2133    R055
2145    R055
6459    R170
647     R017
641     R017
6471    R170
397     R011
635     R017
6837    R179
6831    R179
831     R022
6843    R179
385     R011
2151    R055
6481    R170
2157    R055
855     R022
6496    R170
892     R023
814     R021
1772    R046
Name: unit, dtype: object

In [237]:
top.iloc[2133]

unit                          R055
date_time      2016-08-29 20:00:00
delta_exits                  18810
Name: 2133, dtype: object

In [241]:
mask = (stations_hourly['station'] == 'FULTON')
stations_hourly[mask].head(50)

KeyError: 'station'

In [242]:
mask = (stations_hourly['unit'] == 'R028') & (stations_hourly['delta_exits'] > 1100)
stations_hourly[mask].head(50)


,unit,date_time,delta_exits
1017,R028,2016-08-27 11:00:00,1586.0
1018,R028,2016-08-27 15:00:00,2605.0
1019,R028,2016-08-27 19:00:00,2730.0
1020,R028,2016-08-27 23:00:00,1805.0
1023,R028,2016-08-28 11:00:00,1210.0
1024,R028,2016-08-28 15:00:00,2316.0
1025,R028,2016-08-28 19:00:00,2285.0
1026,R028,2016-08-28 23:00:00,1532.0
1028,R028,2016-08-29 07:00:00,1813.0
1029,R028,2016-08-29 11:00:00,14857.0
